# DO NOT RUN ANY OF THIS UNLESS YOU KNOW WHAT YOU'RE DOING

In [1]:
import pandas as pd
import numpy as np
import random

In [ ]:
### DATASET CLEANING CODE, IGNORE UNLESS YOU KNOW WHAT THIS IS ###


# df = pd.read_csv("./temp/yield_df.csv")
# df = df.drop(df[df["Area"] != "India"].index)
# df = df.drop(columns=["Unnamed: 0", "Year", "Area"])
# df.rename(columns={"hg/ha_yield": "ton/ha_yield", "Item": "crop"}, inplace=True)
# df["ton/ha_yield"] = df["ton/ha_yield"] / 10 / 1000

# df.to_csv("./data/crop.yield.prediction.csv", index=False)

In [ ]:
df = pd.read_csv("./data/crop.yield.prediction.csv")
df

In [ ]:
rainmap = {
    "Jowar": 400,
    "Maize": 750,
    "Potatoes": 1600,
    "Rice": 2375,
    "Soybeans": 750,
    "Sweet potatoes": 1125,
    "Tapioca (Cassava)": 1000,
    "Wheat": 750,
}

In [ ]:
for i, row in df.iterrows():
    df.at[i, "rainfall"] = rainmap[df.at[i, "crop"]] + random.randrange(-100, 100, 1)

In [ ]:
df

In [ ]:
df.to_csv("./data/crop.yield.prediction.csv", index=False)

Jowar prices at https://data.gov.in/resource/variety-wise-daily-market-prices-jowarsorghum-2020

In [ ]:
df = pd.read_csv("./data/market/Jowar.csv")
df

In [ ]:
df = df.sort_values(by=["market", "arrival_date"])

In [ ]:
df.to_csv("./data/market/Jowar.csv", index=False)

In [ ]:
df = pd.read_csv("./data/market/Rice.csv")
# df = df[(df["Centre_Name"] == "MUMBAI") | (df["Centre_Name"] == "NAGPUR")].drop(columns=["Commodity_Name"])

In [ ]:
df = df.ffill()

In [ ]:
df.to_csv("./data/market/Rice.csv", index=False)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
df = pd.read_csv("./data/market/Rice.csv")
df = df[df["Centre_Name"] == "NAGPUR"].drop(columns=["Centre_Name"])
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values("Date", ascending=True)
df

In [ ]:
date_range = pd.date_range(start=df['Date'].min(), end=df['Date'].max(), freq='D')
date_range

In [ ]:
range_df = pd.DataFrame({'Date': date_range})
df = pd.merge(range_df, df, on='Date', how='left')
df['Price'].interpolate(method='linear', inplace=True)
df

In [ ]:
import math

i = 0

def tweak_prices(x):
    global i
    ret = 20 * math.sin(i) + x
    i += math.pi / 7
    return ret

i = 0
plt.figure(figsize=(20, 4))
plt.plot(df["Date"], df["Price"].apply(tweak_prices))

In [ ]:
df["Price"] = df["Price"].apply(tweak_prices)

In [ ]:
df.to_csv("./data/market/Rice.Nagpur.csv", index=False)

In [5]:
df1 = pd.read_csv("./data/crop.rec.csv")

In [5]:
df2 = pd.read_csv("./data/crop.yield.prediction.csv")

In [19]:
df2["season"] = np.nan

In [27]:
np.unique(df1["label"]), np.unique(df2["crop"])

(array(['Maize', 'Rice', 'apple', 'banana', 'blackgram', 'chickpea',
        'coconut', 'coffee', 'cotton', 'grapes', 'jute', 'kidneybeans',
        'lentil', 'mango', 'mothbeans', 'mungbean', 'muskmelon', 'orange',
        'papaya', 'pigeonpeas', 'pomegranate', 'watermelon'], dtype=object),
 array(['Jowar', 'Maize', 'Potatoes', 'Rice', 'Soybeans', 'Sweet potatoes',
        'Tapioca (Cassava)', 'Wheat'], dtype=object))

In [47]:
crops = {
    # yield
    "Jowar": {"season": "Rabi", "average": 0},
    "Potatoes": {"season": "Rabi", "average": 0},
    "Wheat": {"season": "Rabi", "average": 0},
    "Maize": {"season": "Kharif", "average": 0},
    "Rice": {"season": "Kharif", "average": 0},
    "Soybeans": {"season": "Kharif", "average": 0},
    "Sweet potatoes": {"season": "Both", "average": 0},
    "Tapioca (Cassava)": {"season": "Both", "average": 0},
    # rec
    "apple": {"season": "Both", "average": 0},
    "banana": {"season": "Kharif", "average": 0},
    "blackgram": {"season": "Rabi", "average": 0},
    "chickpea": {"season": "Rabi", "average": 0},
    "coconut": {"season": "Kharif", "average": 0},
    "coffee": {"season": "Both", "average": 0},
    "cotton": {"season": "Kharif", "average": 0},
    "grapes": {"season": "Kharif", "average": 0},
    "jute": {"season": "Kharif", "average": 0},
    "kidneybeans": {"season": "Kharif", "average": 0},
    "lentil": {"season": "Rabi", "average": 0},
    "mango": {"season": "Both", "average": 0},
    "mothbeans": {"season": "Rabi", "average": 0},
    "mungbean": {"season": "Kharif", "average": 0},
    "muskmelon": {"season": "Kharif", "average": 0},
    "orange": {"season": "Both", "average": 0},
    "papaya": {"season": "Both", "average": 0},
    "pigeonpeas": {"season": "Kharif", "average": 0},
    "pomegranate": {"season": "Both", "average": 0},
    "watermelon": {"season": "Kharif", "average": 0},
}

In [7]:
for key in crops:
  crops[key]["average"] = df2[df2["crop"] == key]["ton/ha_yield"].mean()

In [7]:
for index, row in df2.iterrows():
    crop = row["crop"]
    yld = row["ton/ha_yield"]
    avg = crops[crop]["average"]
    season = crops[crop]["season"]

    if season == "Both":
        df2["season"][index] = random.choice(["Rabi", "Kharif"])
        continue

    if yld < avg:
        df2["season"][index] = "Kharif" if season == "Rabi" else "Rabi"
    else:
        df2["season"][index] = season

KeyError: 'ton/ha_yield'

In [32]:
df2.to_csv("./data/crop.yield.prediction.csv", index=False)

In [6]:
df1["season"] = np.nan

In [48]:
for index, row in df1.iterrows():
  season = crops[row["label"]]["season"]
  if season == "Both":
    season = random.choice(["Rabi", "Kharif"])
  df1["season"][index] = season

/tmp/ipykernel_13877/3101779683.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["season"][index] = season
/tmp/ipykernel_13877/3101779683.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["season"][index] = season
/tmp/ipykernel_13877/3101779683.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["season"][index] = season
/tmp/ipykernel_13877/3101779683.py:5: SettingWithCopyWarning: 
A value is trying to be set on a c

In [49]:
df1.to_csv("./data/crop.rec.csv", index=False)

In [6]:
import pandas as pd

df = pd.read_csv("./data/market/Bhivandi.Rice.csv")
df = df.drop(columns=["District Name", "Market Name", "Commodity", "Variety", "Grade", "Sl no."])
df.to_csv("./data//market/Bhivandi.Rice.csv",index=False)